2 Strategies:
- create a dataframe client based to understand the behavior of the client and than try track which features better impact in the target 1 (reactivate)
- create a dataframe with all ocurrences of purchase, to try track which features better impact in the target 1 (reactivate)

In [34]:
! pip install imbalanced-learn

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

url = '/home/helenaglx/MLFieldWork/DF_Model.csv'
target = 'target'
# variables to remove based on distinct values 
variables = ['client_create_date','yyyymm', 'item_id', 'net', 'family_code', 'sales_id','time_diff_prec','time_diff_next']

In [22]:
df = pd.read_csv(url)
df.head()

,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,...,sales_channel,net,family_code,group_code,sales_id,n_purchases,sales_net,time_diff_prec,time_diff_next,target
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,...,C,937.94,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,NaN,1126.0,1
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,...,C,641.09,XBXV2AB,XBXV2AB0102,9306_201709,2,1579.03,NaN,1126.0,1
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,...,C,0.00,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,NaN,1126.0,1
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,...,C,1341.14,XBXV1AF,XBXV1AF0201,9306_202010,2,1341.14,1126.0,NaN,0
4,8321,2005-11-15,LE,15500,2,681.26,E,T8,2021-11-01,1657,...,B,190.15,XAEG1AA,XAEG1AA0103,8321_202111,7,190.15,123.0,NaN,0


In [23]:
df_pos = df.copy()
df_pos.drop(variables, axis=1, inplace=True)

In [24]:
df_pos.shape

(355997, 13)

In [25]:
df_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355997 entries, 0 to 355996
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   client_id      355997 non-null  int64  
 1   region         348804 non-null  object 
 2   trade_sector   355997 non-null  int64  
 3   n_employees    355997 non-null  int64  
 4   economic_pot   355997 non-null  float64
 5   eco_pot_class  355997 non-null  object 
 6   risk_cat       355997 non-null  object 
 7   flg_tool       355997 non-null  int64  
 8   sales_channel  355997 non-null  object 
 9   group_code     355997 non-null  object 
 10  n_purchases    355997 non-null  int64  
 11  sales_net      355997 non-null  float64
 12  target         355997 non-null  int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 35.3+ MB


Use the most frequence value for the columns and the max for the target

In [26]:
def most_frequent(x):
    return x.mode().iloc[0] if not x.mode().empty else pd.NA

# Columns to aggregate by most frequent (excluding target)
columns_to_mode = df_pos.columns.drop(['client_id', 'target'])

# Apply mode aggregation to those columns
agg_mode = df_pos.groupby('client_id')[columns_to_mode].agg(most_frequent)

# For target, take the maximum (1 if they ever reactivated)
agg_target = df_pos.groupby('client_id')['target'].max()

# Combine them back into a single dataframe
client_summary = pd.concat([agg_mode, agg_target], axis=1).reset_index()


In [10]:
client_summary

,client_id,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,flg_tool,sales_channel,group_code,n_purchases,target
0,1,VT,22100,2,4528.46,E,3d,0,D,XAEC4AA0101,1,0
1,2,NO,13500,2,4432.42,E,3d,1,D,XAEN2BA0103,2,0
2,3,BA,21100,3,5000.00,E,3d,1,B,XAEP3CC0301,6,0
3,4,MC,22100,4,5000.00,E,3d,0,A,XBCF1AA0101,1,0
4,5,AN,13500,2,4432.42,E,3d,1,C,XAEC3AA0103,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
85481,93253,MI,36100,1,2538.00,E,5d,0,B,XAEC2CA0601,1,0
85482,93254,ME,11000,7,9772.08,D,5d,1,B,XADC1CA0306,1,0
85483,93255,MI,13500,2,4432.42,E,5d,1,B,XAEP3CC0302,1,0
85484,93256,MB,14400,4,1815.72,E,5d,1,B,XAEP3CA0102,1,0


In [27]:
client_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85289 entries, 0 to 85288
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   client_id      85289 non-null  int64  
 1   region         83453 non-null  object 
 2   trade_sector   85289 non-null  int64  
 3   n_employees    85289 non-null  int64  
 4   economic_pot   85289 non-null  float64
 5   eco_pot_class  85289 non-null  object 
 6   risk_cat       85289 non-null  object 
 7   flg_tool       85289 non-null  int64  
 8   sales_channel  85289 non-null  object 
 9   group_code     85289 non-null  object 
 10  n_purchases    85289 non-null  int64  
 11  sales_net      85289 non-null  float64
 12  target         85289 non-null  int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 8.5+ MB


### Make dummies

In [28]:
# Select categorical columns
categorical_cols = ['eco_pot_class', 'risk_cat', 'region', 'sales_channel', 'group_code']

# Apply one-hot encoding
df_dummies = pd.get_dummies(client_summary, columns=categorical_cols, drop_first=True)

# Display the first few rows
df_dummies.head()

,client_id,trade_sector,n_employees,economic_pot,flg_tool,n_purchases,sales_net,target,eco_pot_class_E,risk_cat_1b,...,group_code_XDOT5AA0103,group_code_XDOT5AA0105,group_code_XDOT5AA0106,group_code_XDOT5AA0111,group_code_XDOX6AA0101,group_code_XDOX7AA0601,group_code_XDOX7AD0202,group_code_XDOX7BC0101,group_code_XDOX7DZ9999,group_code_XDOX7ZZ9999
0,1,22100,2,4528.46,0,1,57.56,0,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2,13500,2,4432.42,1,2,477.84,0,True,False,...,False,False,False,False,False,False,False,False,False,False
2,3,21100,3,5000.00,1,6,611.10,0,True,False,...,False,False,False,False,False,False,False,False,False,False
3,4,22100,4,5000.00,0,1,364.44,0,True,False,...,False,False,False,False,False,False,False,False,False,False
4,5,13500,2,4432.42,1,1,275.63,0,True,False,...,False,False,False,False,False,False,False,False,False,False


## Models

In [29]:
df_model = df_dummies.copy()
df_model.drop(['client_id'], axis=1, inplace=True)

In [30]:
from sklearn.model_selection import train_test_split

X = df_model.drop(columns='target')
y = df_model['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [32]:
model = RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94     15070
           1       0.61      0.11      0.19      1988

    accuracy                           0.89     17058
   macro avg       0.75      0.55      0.56     17058
weighted avg       0.86      0.89      0.85     17058

ROC AUC: 0.8530732503848574


### Try balance the data using SMOTE

In [36]:
from sklearn.preprocessing import StandardScaler
# 1. Split your data
X = df_model.drop('target', axis=1)  # Replace 'target' with your label column
y = df_model['target']

# Optional: Scale features if needed (SMOTE benefits from this sometimes)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# 3. Apply SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"Original train shape: {X_train.shape}, Resampled train shape: {X_train_res.shape}")

# 4. Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')  # class_weight is optional here
rf.fit(X_train_res, y_train_res)

# 5. Evaluate
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))

print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

Original train shape: (68231, 1680), Resampled train shape: (120560, 1680)
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.98      0.94     15070
           1       0.51      0.12      0.20      1988

    accuracy                           0.88     17058
   macro avg       0.70      0.55      0.57     17058
weighted avg       0.85      0.88      0.85     17058

ROC AUC Score: 0.8320


### Only with numeric features

In [41]:
df_numerical = client_summary.drop(categorical_cols, axis=1)

X = df_numerical.drop(columns=['target', 'client_id'])
y = df_numerical['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

model = RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92     15070
           1       0.29      0.19      0.23      1988

    accuracy                           0.85     17058
   macro avg       0.60      0.56      0.57     17058
weighted avg       0.83      0.85      0.84     17058

ROC AUC: 0.7998312035450927


In [42]:
print(f'df_numerical shape: {df_numerical.shape}')
df_numerical.info()

df_numerical shape: (85289, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85289 entries, 0 to 85288
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   client_id     85289 non-null  int64  
 1   trade_sector  85289 non-null  int64  
 2   n_employees   85289 non-null  int64  
 3   economic_pot  85289 non-null  float64
 4   flg_tool      85289 non-null  int64  
 5   n_purchases   85289 non-null  int64  
 6   sales_net     85289 non-null  float64
 7   target        85289 non-null  int64  
dtypes: float64(2), int64(6)
memory usage: 5.2 MB


### Explainability
Check the reability of the model (confidence of the model)
https://www.datacamp.com/pt/tutorial/introduction-to-shap-values-machine-learning-interpretability

RIC: Ranking Information Correlation